In [1]:
%load_ext jupyter_black

In [2]:
import json
from glob import glob
from pathlib import Path
from typing import Iterator, NewType, Union
from datetime import datetime
from warnings import warn
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame

from sppp.schema import FeatureCollection

idx: slice = pd.IndexSlice

In [5]:
def iterpaths() -> Iterator[Path]:
    yield from (Path(file) for file in glob("/workspaces/sppp/data/2022/**/*.json"))


def to_midf() -> pd.DataFrame:
    def generate():
        for path in iterpaths():
            with path.open("rb") as f:
                try:
                    fc: FeatureCollection = json.load(f)
                except json.JSONDecodeError:
                    warn(f"json error decoding {path}")
                    continue
                df: GeoDataFrame = GeoDataFrame.from_features(fc["features"])
                df["validTime"] = datetime.strptime(fc["validTime"], "%Y%m%d_%H%M%S %Z")
                yield df

    return pd.concat(generate()).set_index(["validTime", "ID"])


df = to_midf()
df

geometry  \
validTime           ID                                                          
2022-07-20 10:12:39 360483  POLYGON ((-84.86000 36.23000, -84.83000 36.230...   
                    360507  POLYGON ((-83.38000 33.90000, -83.35000 33.900...   
                    360611  POLYGON ((-85.88000 37.17000, -85.84000 37.170...   
                    360845  POLYGON ((-78.03000 45.45000, -78.00000 45.450...   
                    360923  POLYGON ((-98.74000 35.52000, -98.71000 35.520...   
...                                                                       ...   
2022-07-20 01:46:39 358602  POLYGON ((-109.32000 30.84000, -109.28000 30.8...   
                    358603  POLYGON ((-110.02000 30.74000, -109.98000 30.7...   
                    358604  POLYGON ((-85.58000 30.12000, -85.57000 30.110...   
                    358605  POLYGON ((-85.52000 29.43000, -85.45000 29.430...   
                    358606  POLYGON ((-83.49000 29.15000, -83.47000 29.140...   

                           MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime           ID                                           
2022-07-20 10:12:39 360483   1487    852   -22    32.5      61   
                    360507   2172   1459   -25    36.7      73   
                    360611   2101   1437   -29    22.8      95   
                    360845    723    372   -87    19.5     101   
                    360923    455     28     0    33.5     188   
...                           ...    ...   ...     ...     ...   
2022-07-20 01:46:39 358602   4181   2374   -43    39.1      47   
                    358603   4011   2452   -22    38.1       6   
                    358604   3169   2561    -4    22.2      27   
                    358605   3697   2857    -2    22.4      14   
                    358606   3896   3089    -1    14.2       0   

                           MEANWIND_1-3kmAGL  MESH VIL_DENSITY FLASH_RATE  \
validTime           ID                                                      
2022-07-20 10:12:39 360483              20.7  0.12        0.97          1   
                    360507              20.5  0.30        2.23          4   
                    360611              16.9  0.00        1.10          0   
                    360845              25.1  0.63        2.25          1   
                    360923              27.1  0.04        0.83          4   
...                                      ...   ...         ...        ...   
2022-07-20 01:46:39 358602               6.2  0.79        1.73          5   
                    358603               5.8  0.62        1.26         24   
                    358604              14.2  0.11        0.57          0   
                    358605              13.2  0.19        0.46          0   
                    358606              12.6  0.04        0.63          0   

                            ...              MAXRC_ICECF WETBULB_0C_HGT PWAT  \
validTime           ID      ...                                                
2022-07-20 10:12:39 360483  ...  0731Z 0.13/min (strong)           10.7  1.7   
                    360507  ...                      N/A           12.7  2.0   
                    360611  ...  0906Z 0.09/min (strong)           11.6  1.7   
                    360845  ...    0831Z 0.02/min (weak)           11.7  1.6   
                    360923  ...                      N/A           14.4  1.4   
...                         ...                      ...            ...  ...   
2022-07-20 01:46:39 358602  ...                      N/A           12.2  1.5   
                    358603  ...                      N/A           11.6  1.6   
                    358604  ...                      N/A           13.5  1.9   
                    358605  ...     0131Z 0.0/min (weak)           14.6  2.1   
                    358606  ...     0011Z 0.0/min (weak)           14.6  2.2   

                           CAPE_M10M30  LJA SIZE         AVG_BEAM_HGT  \
validTime           ID                  